### Introduction
This notebook illustrates and automates the Continuous Deployment process for bringing the popular open-source large language models backend [llama.cpp](https://github.com/ggerganov/llama.cpp) into SAP AI Core. Running LLaMa 3, Phi3, Mistral, Mixtral, LlaVA, and other [supported models in llama.cpp](https://github.com/ggerganov/llama.cpp) in SAP AI Core through BYOM(Bring Your Own Model) approach. <br/>

### Prerequisites
Before running this notebook, please assure you have perform the [Prerequisites](../../README.md)<br/><br/>

If the configuration of llama.cpp scenario is created through SAP AI Launchpad instead of running [00-init-config.ipynb](../00-init-config.ipynb), please manually update the configuration_id in [env.json](env.json)
```json
{
    "configuration_id": "<YOUR_CONFIGURATION_ID_OF_LLAMA.CPP_SCENARIO>",
    "deployment_id": "<WILL_BE_UPDATED_BY_THIS_NOTEBOOK>"
}
```
 
### The high-level flow of this Continuous Deployment process:
- Build a custom docker image adapted for SAP AI Core<br/>
- Push the docker image to docker hub<br/>
- Connect to SAP AI Core via SDK<br/>
- Create a deployment<br/>
- Check the status and logs of the deployment<br/>


#### 1.Build a custom docker image adapted for SAP AI Core
Please refer to [Dockerfile](Dockerfile) for details.

In [ ]:
%%sh
# 0.Login to docker hub
docker login -u <YOUR_DOCKER_USER> -p <YOUR_DOCKER_ACCESS_TOKEN>

# 1.Build the docker image
docker build \
		--platform=linux/amd64 \
		-t docker.io/<YOUR_DOCKER_USER>/llama.cpp-server:ai-core .

#### 2.Push the docker image to docker hub

In [ ]:
%%sh
# 2.Push the docker image to docker hub to be used by deployment in SAP AI Core
docker push docker.io/<YOUR_DOCKER_USER>/llama.cpp-server:ai-core

#### 3.Initiate an SAP AI Core SDK client
- resource_group loaded from [../config.json](../config.json)
- ai_core_sk(service key) loaded from [../config.json](../config.json)

In [1]:
import requests, json, time, datetime
from datetime import datetime
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

In [5]:
# load the configuration from ../config.json 
with open("../config.json") as f:
    config = json.load(f)

resource_group = config.get("resource_group", "default")
print( "resource group: ", resource_group)

resource group:  oss-llm


In [3]:
# Initiate an AI Core SDK client with the information of service key
ai_core_sk = config["ai_core_service_key"]
base_url = ai_core_sk.get("serviceurls").get("AI_API_URL") + "/v2/lm"
client = AICoreV2Client(base_url=ai_core_sk.get("serviceurls").get("AI_API_URL")+"/v2",
                        auth_url=ai_core_sk.get("url")+"/oauth/token",
                        client_id=ai_core_sk.get("clientid"),
                        client_secret=ai_core_sk.get("clientsecret"),
                        resource_group=resource_group)

In [4]:
# Prepare the http header which will be used later through request.
token = client.rest_client.get_token()
headers = {
    "Authorization": token,
    "ai-resource-group": resource_group,
    "Content-Type": "application/json",
}

#### 4.Create a deployment for llama.cpp scenario
To create a deployment in SAP AI Core, it requires the corresponding resource_group and configuration_id
- resource_group loaded from [../config.json](../config.json)
- configuration_id of  loaded from [env.json](env.json)

In [6]:
# resource_group: The target resource group to create the deployment
# configuration_id: The target configuration to create the deployment, which is created in ../00-init-config.ipynb 
with open("./env.json") as f:
    env = json.load(f)

configuration_id = env["configuration_id"]
print("configuration id:", configuration_id)

configuration id: 33a87ba9-48e1-431c-b963-282665428ca4


**Helper function**
- get the current UTC time in yyyy-mm-dd hh:mm:ss format, to be used to filter deployments logs

In [7]:
# Helper function to get the current time in UTC, used to filter deployments logs
def get_current_time():  
    current_time = datetime.utcnow()
    # Format current time in the desired format
    formatted_time = current_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    return formatted_time

**Helper function**
- Write back the configuration value back to configuration json file

In [8]:
# Helper function to write back the configuration value back to configuration json file
def update_json_file(file_path, key, value):
    # Load the JSON configuration file
    with open(file_path, 'r') as file:
        config = json.load(file)

    # Update the value
    config[key] = value

    # Write the updated configuration back to the file
    with open(file_path, 'w') as file:
        json.dump(config, file, indent=4)
        print(f"{file_path} updated. {key}: {value}")

**Create a deployment for llama.cpp in SAP AI Core**
- configuration_id
- resource_group
<br/><br/>
The created deployment id will be written back to [env.json](env.json), which will be used in
- [02-llama.cpp.ipynb](02-llama.cpp.ipynb)to test the inference of open-source llms with llama.cpp server in SAP AI Core
- [04-cleanup.ipynb](04-cleanup.ipynb) to stop and delete the deployment and clean up the resource.

In [9]:
# Create a Deployment in SAP AI Core
print("Creating deployment.")
response = client.deployment.create(
    configuration_id=configuration_id,
    resource_group=resource_group
)

# last_check_time will be used to check the deployment status continuously afterwards
# set initial last_check_time right after creating deployment
last_check_time = get_current_time()
deployment_start_time = datetime.now()

deployment_id = response.id
status = response.status
update_json_file("env.json", "deployment_id", deployment_id)
print("Deployment Result:\n", response.__dict__)

Creating deployment.
env.json updated. deployment_id: d2b3f7ffbde78b38
Deployment Result:
 {'id': 'd2b3f7ffbde78b38', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


#### 5.Check the status and logs of the deployment

In [10]:
print("5.Checking deployment status.")
deployment_url = f"{base_url}/deployments/{deployment_id}"
deployment_log_url = f"{deployment_url}/logs?start="
interval_s = 20

while status != "RUNNING" and status != "DEAD":
    current_time = get_current_time()
    #check deployment status
    response = requests.get(url=deployment_url, headers=headers)
    resp = response.json()
    
    status = resp['status']
    print(f'...... Deployment Status at {current_time}......', flush=False)
    print(f"Deployment status: {status}")

    #retrieve deployment logs
    response_log = requests.get(url=f"{deployment_log_url}{last_check_time}", headers=headers)
    last_check_time = current_time
    print(f"Deployment logs: {response_log.text}")

    # Sleep for 60 secs to avoid overwhelming the API with requests
    time.sleep(interval_s)

deployment_end_time = datetime.now()
duration_in_min = (deployment_end_time - deployment_start_time) / 60

if status == "RUNNING":
    print("Deployment is up and running now!")
else:
    print(f"Deployment {deployment_id} failed!")   

print(f"Deployment duration: {duration_in_min} mins")

5.Checking deployment status.
...... Deployment Status at 2024-03-27T09:39:40.825379Z......
Deployment status: UNKNOWN
Deployment logs: {
  "error": {
    "code": "05011000",
    "message": "DeploymentNotFoundError: Deployment d2b3f7ffbde78b38 not found.",
    "target": "/api/v4alpha/deployments/d2b3f7ffbde78b38/logs"
  }
}

...... Deployment Status at 2024-03-27T09:40:03.404014Z......
Deployment status: UNKNOWN
Deployment logs: {
  "error": {
    "code": "05011000",
    "message": "DeploymentNotFoundError: Deployment d2b3f7ffbde78b38 not found.",
    "target": "/api/v4alpha/deployments/d2b3f7ffbde78b38/logs"
  }
}

...... Deployment Status at 2024-03-27T09:40:25.986644Z......
Deployment status: UNKNOWN
Deployment logs: {
  "data": {
    "result": []
  }
}

...... Deployment Status at 2024-03-27T09:40:56.702019Z......
Deployment status: PENDING
Deployment logs: {
  "data": {
    "result": []
  }
}

...... Deployment Status at 2024-03-27T09:41:31.801897Z......
Deployment status: PENDING